## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd

In [2]:
credit_debts = pd.read_csv('data.csv')
print(credit_debts.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



In [3]:
display(credit_debts.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 1. Предобработка данных

In [4]:
# просмотр уникальных значений столбцов (кроме стажа и дохода, где почти все значения будут уникальны)
print("УНИКАЛЬНЫЕ ЗНАЧЕНИЯ СТОЛБЦОВ")
print()
print("Дети: ", end='')
print(credit_debts.children.unique())
print()
print("Возраст: ", end='')
print(credit_debts.dob_years.unique())
print()
print("Образование: ", end='')
print(credit_debts.education.unique())
print()
print("Идентификатор образования: ", end='')
print(credit_debts.education_id.unique())
print()
print("Семейное положение: ", end='')
print(credit_debts.family_status.unique())
print()
print("Идентификатор семейного положения: ", end='')
print(credit_debts.family_status_id.unique())
print()
print("Пол: ", end='')
print(credit_debts.gender.unique())
#print(len(credit_debts[credit_debts.gender == 'XNA']))
print()
print("Тип дохода: ", end='')
print(credit_debts.income_type.unique())
print()
print("Была ли просрочка по кредиту: ", end='')
print(credit_debts.debt.unique())
print()
print("Цель кредита: ", end='')
print(credit_debts.purpose.unique())

УНИКАЛЬНЫЕ ЗНАЧЕНИЯ СТОЛБЦОВ

Дети: [ 1  0  3  2 -1  4 20  5]

Возраст: [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

Образование: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Идентификатор образования: [0 1 2 3 4]

Семейное положение: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Идентификатор семейного положения: [0 1 2 3 4]

Пол: ['F' 'M' 'XNA']

Тип дохода: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Была ли просрочка по кредиту: [0 1]

Цель кредита: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'


### Вывод

1.  По общей информации видно, что есть пропуски в столбцах "days_employed" и "total_income" (в одном и том же количестве).
2. В столбце "days_employed" отрицательные значения. Вероятно, для анализа зависимости просрочки от семейного положения и количества детей, этот параметр не нужен, тем более что в таблице есть уровень дохода. Кроме того, какой смысл в указании стажа для пенсионеров (столбец "тип дохода"), например? Столбец можно выкинуть.
3. -1 ребенок - скрытый пропуск. Можно будет заменить на медиану или среднее. 20 детей - похоже на выброс, но надо это проверить.
4. Возраст - 0, видимо, скрытый пропуск. Можно будет заменить на медиану или среднее.
5. Дубликаты в столбце "education" ('среднее' и "СРЕДНЕЕ" ит.п.). Можно исправить на стадии ручной работы с дубликатами. 
6. Идентификатор образования - бесполезный параметр (удобнее пользоваться значениями "высшее", "среднее" итд. из предыдущего столбца). Этот столбец можно выкинуть.
7. Семейное положение - имеет смысл свести к двум категориям (состоит или не состоит в браке).
8. Идентификатор семейного положения после категоризации данных предыдущего столбца станет бесполезным параметром, его можно выкинуть.
9. Пол: есть одна строка со значением XNA. Видимо, это значит, что пол не указан. Можно удалить.
10. Тип дохода нужно сгруппировать по категориям. Компаньонов объединить с предпринимателями, сотрудников с госслужащими, безработных со студентами и группой "в декрете".
11. Уникальных значений для цели кредита достаточно много. Поэтому для категоризации нужно будет выяснить, какие леммы встерчаются там чаще всего.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# Для начала выкинем ненужные столбцы
credit_debts.drop(['days_employed', 'education_id', 'family_status_id'], axis = 'columns', inplace = True)

# Чтобы понять, можно ли удалить пропуски в столбце "total_income" характерными значениями без ущерба для анализа,
#проверим, есть ли связь между просрочкой кредита
# и отсутствием информации о доходе. Для этого сравним долю просроченных кредитов по выборке в целом с
# долей просроченных кредитов среди тех, о чьих доходах нет данных

#посчитаем долю просроченных кредитов по всей выборке
bad_debts = credit_debts.debt.sum()
bad_debts_part = bad_debts / len(credit_debts)
print("Доля просроченных кредитов для всей выборки: ", end="")
print("{:.2%}".format(bad_debts_part))
print()

# посчитаем число ячеек в 'total_income' с пропущенными значениями
rows_with_nans = credit_debts.total_income.isna().sum()
# посчитаем долю просроченных кредитов среди тех, о чьих доходах нет данных
bad_debts_in_nans = len(credit_debts[(credit_debts.total_income.isna() == True) & (credit_debts.debt == 1)])
bad_debts_in_nans_part = bad_debts_in_nans / rows_with_nans
print("Доля просроченных кредитов среди тех, о чьих доходах нет данных: ", end="")
print('{:.2%}'.format(bad_debts_in_nans_part))
print()

# узнаем, сколько строк, где указано 20 детей и много ли там неоплаченных долгов
print("Строк, где якобы 20 детей: ", end="")
print(credit_debts[credit_debts.children == 20]['children'].count())
print()
# Доля просрочки для "20 детей"
print("Доля просрочки для заемщиков, у которых 20 детей: ", end="")
twenty_kids = credit_debts[(credit_debts.children == 20)]
print('{:.2%}'.format(twenty_kids.debt.mean()))
print()

#удалим пропуски
credit_debts.dropna(subset = ['total_income'], inplace = True)
credit_debts.drop(credit_debts[credit_debts.children == 20].index,  inplace = True)
credit_debts.drop(credit_debts[credit_debts.gender == 'XNA'].index, inplace = True)
credit_debts = credit_debts.reset_index(drop=True)
# проверим, что получилось
print(credit_debts.info())
print()
# заменим в "children" -1 на медиану; в возрасте 0 - на медиану
credit_debts['children'] = credit_debts['children'].replace(-1, credit_debts.children.median())
credit_debts['dob_years'] = credit_debts['dob_years'].replace(0, credit_debts.dob_years.median())
# проверим, что получилось
print(credit_debts[credit_debts.children == -1])
print()
print(credit_debts[credit_debts.dob_years == 0])

Доля просроченных кредитов для всей выборки: 8.09%

Доля просроченных кредитов среди тех, о чьих доходах нет данных: 7.82%

Строк, где якобы 20 детей: 76

Доля просрочки для заемщиков, у которых 20 детей: 10.53%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19283 entries, 0 to 19282
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       19283 non-null  int64  
 1   dob_years      19283 non-null  int64  
 2   education      19283 non-null  object 
 3   family_status  19283 non-null  object 
 4   gender         19283 non-null  object 
 5   income_type    19283 non-null  object 
 6   debt           19283 non-null  int64  
 7   total_income   19283 non-null  float64
 8   purpose        19283 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.3+ MB
None

Empty DataFrame
Columns: [children, dob_years, education, family_status, gender, income_type, debt, total_income, purpose]
Inde

### Вывод

1. Пропусков в столбце о доходе было столько же, сколько в столбце о стаже. Очевидно, это одни и те же строки, и, видимо, это означает, что не все заемщики предоставляют справки о доходах с места работы. Доля просроченных кредитов среди тех, для кого данные о доходах не указаны, принципиально не отличается от доли просроченных кредитов по выборке в целом. Строки с пропусками удалены.
2. "20 детей" окзались не единичным явлением. Всего 76 таких случаев, при том, что значений от 6 до 19 детей в выборке нет. Хорошего объяснения этому нет. Для заполнения пропуска "20" выглядит странновато. Доля просрочки выше среднего по выборке, но 76 строк - это очень незначительная часть выборки, поэтому все строки, где 20 детей, удалены.
3. -1 в столбце children и 0 в столбце dob_years заменены на медианные значения, потому что медиана в целом точнее харктеризует выборку, чем среднее арифметическое.

### Замена типа данных

In [6]:
# В дробных значениях для числа рублей,
# измеряемого сотнями тысяч, скорее всего, нет нужды
# Поэтому цифры после точки можно отбросить.
# Для замены вещественных значений на целочисленные воспользуемся методом astype


credit_debts['total_income'] = credit_debts['total_income'].astype('int')

#проверим результат замены
print(credit_debts.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19283 entries, 0 to 19282
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   children       19283 non-null  int64 
 1   dob_years      19283 non-null  int64 
 2   education      19283 non-null  object
 3   family_status  19283 non-null  object
 4   gender         19283 non-null  object
 5   income_type    19283 non-null  object
 6   debt           19283 non-null  int64 
 7   total_income   19283 non-null  int32 
 8   purpose        19283 non-null  object
dtypes: int32(1), int64(3), object(5)
memory usage: 1.3+ MB
None


### Вывод

Столбцу 'total_income' присвоен тип int.

### Обработка дубликатов

In [7]:
# поиск повторяющихся строк
print(credit_debts.duplicated().sum())
print()


# замена заглавных букв на строчные в столбце 'education'
credit_debts.education = credit_debts.education.str.lower()
# проверка результата
print(credit_debts.education.unique())

0

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


### Вывод

Дубликатов строк не обнаружено. В столбце "education" дубликаты, связанные с использованием разных регистров, устранены с помощью метода str.lower().

### Лемматизация

In [8]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# Получим текст для лемматизации с помощью конкатенации
# Создаем пустой текст
purpose_words = ""
# Цикл берет список уникльных значений столбца purpose,
# создает список слов, входящих в каждое уникальное значение методом split,
# внутри него другой цикл записывает каждое слово каждого уникального значения в пустой текст
for purpose in credit_debts.purpose.unique():
    purpose_split = purpose.split(" ")
    for item in purpose_split:
        purpose_words = purpose_words + item + " "

# Лемматизируем текст
lemmas = m.lemmatize(purpose_words)
# Подсчитываем, в каком количестве уникальных значений встречается каждая лемма, и выодим результат
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, ' \n': 1})


### Вывод

Из имен существительных чаще всего встречается "покупка", "недвижимость", "автомобиль", "образование", "жилье". Отглагольные существительные и другие существительные, обозначающие действия ("строительство", "приобретение", "получения"), не подходят для обозначения целей кредита: в этом случае прокупка жилья и покупка автомобиля попали бы в одну категорию. Поэтому для категорий нужно выбрать существительные, обозначающие виды товаров и услуг. Это леммы "недвижимость", "жилье" (объединить в одну категорию), "автомобиль", "образование" и "свадьба". Судя по набору лемм, если выделить эти категории, то в категорию "другое" ничего не попадет.

### Категоризация данных

In [9]:
# Категоризацию начнем с изменения названий столбцов.
# Для задания имеет значение, состоит ли заемщик в браке. Поэтому
# мы будем объединять значения "женат | замужем" со значением "гражданский брак" в значение "состоит в браке";
# остальные значения - в "не состоит в браке". Названием соответствующего столбца изменим на 'is_married'
#Также поменяем название столбца "dob_years" на "age" и "purpose" на "credit_purpose" (такие названия понятнее)

credit_debts.set_axis(['children', 'age', 'education', 'is_married', 'gender', 'income_type', 'debt', 'total_income', 'credit_purpose'], axis = 'columns', inplace = True)


# Для категоризации данных в столбце 'is_married' создадим функцию

def married(column):
    if column == 'женат / замужем' or column == "гражданский брак":
        return "состоит в браке"
    else:
        return "не состоит в браке"
    
# Применим эту функцию к столбцу с помощью метода apply
    
credit_debts.is_married = credit_debts.is_married.apply(married)


# Функция для категоризации данных в столбце 'income_type'

def income(column):
    if column == 'сотрудник' or column == "госслужащий":
        return "заработная плата"
    elif column == "предприниматель" or column == "компаньон":
        return "доходы от предпринимательской деятельности"
    else:
        return "пособия и социальные выплаты"

# Применение функции к столбцу
credit_debts.income_type = credit_debts.income_type.apply(income)

In [10]:
# Изменим значения в столбце credit_purpose

for row in range(len(credit_debts)):
    txt = credit_debts['credit_purpose'][row].lower()
    if "недвижимост" in txt:
        credit_debts['credit_purpose'][row] = 'недвижимость'
    elif 'жиль' in txt:
        credit_debts['credit_purpose'][row] = 'недвижимость'
    elif "образован" in txt:
        credit_debts['credit_purpose'][row] = 'образование'
    elif "свадьб" in txt:
        credit_debts['credit_purpose'][row] = 'свадьба'

    elif 'автомоб' in txt:
        credit_debts['credit_purpose'][row] = 'автомобиль'  
    else:
        credit_debts['credit_purpose'][row] = "другое"

In [11]:
print(credit_debts.credit_purpose.value_counts())
print()
print(credit_debts.income_type.value_counts())
print()
print(credit_debts.is_married.value_counts())

недвижимость    9727
автомобиль      3881
образование     3584
свадьба         2091
Name: credit_purpose, dtype: int64

заработная плата                              11285
доходы от предпринимательской деятельности     4558
пособия и социальные выплаты                   3440
Name: income_type, dtype: int64

состоит в браке       14820
не состоит в браке     4463
Name: is_married, dtype: int64


### Вывод

Значения столбца распределились по четырем категориям. В "другое" ничего не попало, это подтверждает вывод, сделанный после лемматизации. При этом, около половины кредитов связано с недвижимостью, а на автомобили и образование берут примерно одинаковое количество кредитов. Вероятно, объяснение банальное: с одной стороны, чем дороже предмет интереса заемщика, тем скорее ему понадобится кредит для его приобретения, а с другой - недвижимость - более насущная необходимость, чем автомобиль, образование и тем более чем слишком дорого стоящая свадьба.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
# Посчитаем средний показатель по столбцу "debt" для групп заемщиков с разным количеством детей,
# сгруппировав данные по столбцу "children". Поскольку значения в столбце 'debt' - нули и единицы,
# у нас получится доля должников для каждой группы. Чтобы оценить репрезентативность
# выборки каждой группы, применим метод count к столбцу "children".
# Столбцы получившейся таблицы переименуем в "доля должников" и "человек в выборке"
children_and_debts = credit_debts.groupby('children').agg({'debt': ['mean'], 'children': ['count']})
children_and_debts.set_axis(['доля должников', 'человек в выборке'], axis = 'columns', inplace = True)

print("Процент должников в группах с разным количеством детей:")
print()
print (children_and_debts)

Процент должников в группах с разным количеством детей:

          доля должников  человек в выборке
children                                   
0               0.074728              12753
1               0.093944               4343
2               0.095624               1851
3               0.074830                294
4               0.088235                 34
5               0.000000                  8


### Вывод

В среднем у должников детей больше, чем у тех, кто платит в срок. Среди бездетных заемщиков, примерно 7.5 % - не платят в срок. Среди тех, у кого есть один ребенок - почти 9.4%. Среди тех, у кого два ребенка, должников примерно столько же - 9.5%. Дальше тенденция ломается: у многодетных проблем с долгами меньше, чем у семей с 1-2 детьми. Среди родителей троих детей процент должников такой же, как среди бездетных - 7.5% Когда у человека детей четверо, доля должников меньше 9%. Однако всего людей с четырьмя детьми в этой выборке 34  - возможно, это не очень репрезентативно. Среди тех, у кого пятеро детей, должников нет вообще. Впрочем, таких людей в выборке всего 8 человек. Поэтому сложно делать выводы. Таким образом, главный вывод в том, что самая проблемная группа - те, у кого 1 или 2 ребенка. Возможно, более состоятельные люди чаще становятся многодетными, а у бездетных меньше расходов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
# Создадим сводную таблицу по состоящим и не состоящим в браке мужчинам и женщинам,
# где в качестве значений укажем долю должников
print("Доля просроченной задолжности в зависимости от семейного положения для женщин и мужчин:")
print()
family_and_debts = credit_debts.pivot_table(index = ['gender'], columns = ['is_married'], values = ['debt'], aggfunc = 'mean')
family_and_debts.set_axis(["должники не в браке", "должники в браке"], axis = 'columns', inplace = True)
print(family_and_debts)
print()

# Отдельно исследуем должников среди одиноких матерей и одиноких отцов. Создадим таблицу по тем, у кого
# есть дети:
having_children = credit_debts[credit_debts.children > 0]
# И сделаем по ней сводную таблицу, такую же, как мы сделали по основной таблице
print("Доля просроченной задолжности в зависимости от семейного положения для женщин и мужчин с детьми:")
print()
single_parents = having_children.pivot_table(index = ['gender'], columns = ['is_married'], values = ['debt'], aggfunc = 'mean')
single_parents.set_axis(["должники не в браке", "должники в браке"], axis = 'columns', inplace = True)
print(single_parents)

Доля просроченной задолжности в зависимости от семейного положения для женщин и мужчин:

        должники не в браке  должники в браке
gender                                       
F                  0.064203          0.072074
M                  0.140945          0.092984

Доля просроченной задолжности в зависимости от семейного положения для женщин и мужчин с детьми:

        должники не в браке  должники в браке
gender                                       
F                  0.080622          0.089947
M                  0.158996          0.095784


### Вывод

Среди женатых мужчин должников 9%, заметно меньше, чем среди холостых - 14 %(!). Среди замужних женщин - 7%, среди незамужних - 6%. Вывод: если заемщик мужчина, имеет смысл обратить внимание на его семейное положение. Наличие детей у одиночек несколько увеличивает вероятность просрочки кредита, так среди матерей-одиночек доля просроченных кредитов составляет 8%. Среди одиноких отцов - 16%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
# Сначала вычислим медианный доход по выборке в целом
print('Медианный доход: ', end='')
income_median = credit_debts.total_income.median()
print(int(income_median))
print()
print()
# Теперь посмотрим как доля должников и медианный доход менябтся в зависисмости от типа дохода
print('Типы дохода, их доли должников и медианные доходы')
print()
income_type_and_debts = credit_debts.groupby('income_type').agg({'debt': ['mean'], 'total_income' : ['median']})
income_type_and_debts.set_axis(['доля должников', "медианный доход"], axis = 'columns', inplace=True)
print(income_type_and_debts)

Медианный доход: 145017


Типы дохода, их доли должников и медианные доходы

                                            доля должников  медианный доход
income_type                                                                
доходы от предпринимательской деятельности        0.075472           172450
заработная плата                                  0.091803           143289
пособия и социальные выплаты                      0.053198           118287


## Вывод

Наибольший процент просрочки - у заемщиков, чей доход - заработная плата (9%). Их медианный доход практически такой же, как медианный доход по выборке в целом. Предприниматели побогаче и немного реже имеют просрочку (7%), однако самые бедные (пенсионеры, студенты ит.п.) выплачивают вовремя чаще всех (5% просроченных кредитов).

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
# Сгруппируем данные по столбцу credit_purpose и вычислим долю должников
print("Доля должников среди заемщиков в зависимости от цели кредита")
print()
targets_and_debts = credit_debts.groupby('credit_purpose')['debt'].mean()
print(targets_and_debts)
print()
# Объединим в сводной таблице данные по типу доходов и цели кредита

print("Доля должников среди заемщиков с разными типами дохода в зависимости от цели кредита")
print()
targets_income_type_debts = credit_debts.pivot_table(index = ['income_type'], columns = ['credit_purpose'], values = ['debt'], aggfunc = 'mean')
print(targets_income_type_debts)

Доля должников среди заемщиков в зависимости от цели кредита

credit_purpose
автомобиль      0.094048
недвижимость    0.073301
образование     0.092076
свадьба         0.074127
Name: debt, dtype: float64

Доля должников среди заемщиков с разными типами дохода в зависимости от цели кредита

                                                 debt               \
credit_purpose                             автомобиль недвижимость   
income_type                                                          
доходы от предпринимательской деятельности   0.080460     0.069554   
заработная плата                             0.110407     0.082884   
пособия и социальные выплаты                 0.061625     0.045936   

                                                                  
credit_purpose                             образование   свадьба  
income_type                                                       
доходы от предпринимательской деятельности    0.075829  0.093418  
заработная плата    

### Вывод

Процент просроченных кредитов, полученных для приобретения автомобиля и получения образования, несколько выше, чем для кредитов, связанных с недвижимостью и свадьбами. Заметно выше среднего процент должников среди тех, кто берет кредит на автомобиль и образование, чьим доходом при этом является заработная плата.

### Шаг 4. Общий вывод

В целом исследование данной выборки не выявило однозначной зависимости возврата кредита от какого-то одного фактора, однако стоит обратить внимание на сочетание таких факторов, как тип дохода и цель кредита. Средний процент просроченных кредитов по всей выборке - 8%. Если кредит берется на автомобиль или образование, а берет его наемный служащий, то вероятность просрочки выше - 10-11%. Также к "группе риска" можно отнести неженатых мужчин. В этой группе доля должников 14%, а если у них есть дети, то 16%.